<a href="https://colab.research.google.com/github/aruaru0/kaggle_h_and_m/blob/main/H%26M%E3%82%AB%E3%82%B9%E3%82%BF%E3%83%9E%E3%83%BC%E5%88%A5%E5%95%86%E5%93%81%E4%B8%80%E8%A6%A7%E4%BD%9C%E6%88%90%EF%BC%88%E6%9C%80%E7%B5%82%E6%9C%88%EF%BC%89.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from collections import defaultdict
import math
from tqdm import tqdm
import json

from gensim.models import word2vec
import multiprocessing
cpu = multiprocessing.cpu_count()

## ファイル読み込み

In [5]:
df  = pd.read_csv('/content/drive/MyDrive/datas/h_and_m/transactions_train.csv',
                            usecols= ['t_dat', 'customer_id', 'article_id'], 
                            dtype={'article_id': 'string', 't_dat': 'string', 'customer_id': 'string'})

df['t_dat'] = pd.to_datetime(df['t_dat'])
#last_ts = df['t_dat'].max()
df.shape

(31788324, 3)

In [6]:
last = df[df['t_dat'] >= datetime(2020,9,1,0,0)]
last.head()

,t_dat,customer_id,article_id
30990055,2020-09-01,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,0777148006
30990056,2020-09-01,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,0835801001
30990057,2020-09-01,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,0923134005
30990058,2020-09-01,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,0865929003
30990059,2020-09-01,0005ed68483efa39644c45185550a82dd09acb07622acb...,0863646004


In [7]:
custdf = last.groupby('customer_id').agg({"article_id": list}).reset_index()

In [8]:
custdf.index  = custdf['customer_id']
custdf

,customer_id,article_id
customer_id,,
00000dbacae5abe5e23885899a1fa44253a17956c6d1c3d25f88aa139fdfc657,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,[0568601043]
000058a12d5b43e67d225668fa1f8d618c13dc232df0cad8ffe7ad4a1091e318,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,[0794321007]
0000757967448a6cb83efb3ea7a3fb9d418ac7adf2379d8cd0c725276a467a2a,0000757967448a6cb83efb3ea7a3fb9d418ac7adf2379d...,"[0719530003, 0448509014]"
000172a9c322560c849754ffbdfdb2180d408aa7176b943f957804686be8e1f0,000172a9c322560c849754ffbdfdb2180d408aa7176b94...,"[0685814001, 0685814001, 0685814001]"
0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37e011580a479e80aa94,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,"[0777148006, 0835801001, 0923134005, 086592900..."
...,...,...
fffd0248a95c2e49fee876ff93598e2e20839e51b9b7678aab75d9e8f9f3c6c8,fffd0248a95c2e49fee876ff93598e2e20839e51b9b767...,"[0509091057, 0859737002, 0573085028, 074568600..."
fffd870c6324ad3bda24e4d6aeae221c199479086bfdfd0eb40a3065971d0e01,fffd870c6324ad3bda24e4d6aeae221c199479086bfdfd...,"[0750423010, 0761269001]"
fffef3b6b73545df065b521e19f64bf6fe93bfd450ab20e02ce5d1e58a8f700b,fffef3b6b73545df065b521e19f64bf6fe93bfd450ab20...,"[0910601003, 0873276003, 0905365002, 090536500..."


In [9]:
cust_dict = custdf.drop('customer_id', axis=1).to_dict()
cust_dict = cust_dict['article_id']

In [10]:
with open('customer_articles_202009.json','w') as f:
  json.dump(cust_dict, f)

## item2vec作成

In [11]:
from tqdm import tqdm
# テキスト形式にしｔ保存
with open('items.txt', 'w') as f:
    for e in tqdm(cust_dict) :
      l = " ".join(cust_dict[e])
      f.write(l + '\n')

100%|██████████| 189510/189510 [00:00<00:00, 590825.64it/s]


In [14]:
sentences = word2vec.LineSentence('items.txt')

# word2vecのモデルを構築
#     sg: 1ならskip-gram(cbowではなく)
#     size: 単語ベクトルの次元数
#     window: ウィンドウサイズ(今回は1行に含まれる全ての単語を対象とするため大きな値を設定)
#     hs: 0ならネガティブサンプリング(1なら階層的ソフトマックス)
#     negative: ネガティブサンプリングする単語数(結果への影響が予測できなかったのでデフォルトの5に設定)
#     seed: 乱数シード(結果を安定させたいので固定値を設定)
#model = word2vec.Word2Vec(sentences, sg=1, size=100, window=100000, hs=0, negative=5, seed=42)
model = word2vec.Word2Vec(sentences,  size=256, window=100000, seed=42, min_count=1, workers=cpu)

In [15]:
model.save("word2vec.model")

In [17]:
!cp *.json *.txt word2vec*  /content/drive/MyDrive/datas/h_and_m/trend202009

In [21]:
model.wv.most_similar(['0802871001',
  '0827428001',
  '0865076001'], topn=10)


[('0826646009', 0.9663570523262024),
 ('0824995006', 0.963238000869751),
 ('0778064038', 0.9616572260856628),
 ('0768931006', 0.9572802186012268),
 ('0754005001', 0.9561100006103516),
 ('0820484003', 0.9558284282684326),
 ('0881425002', 0.9556455612182617),
 ('0516000001', 0.9549257755279541),
 ('0825719002', 0.9548330307006836),
 ('0622969001', 0.9534097909927368)]

In [31]:
i = 0
for e in cust_dict :  
  ret = model.wv.most_similar(cust_dict[e], topn=4)
  print(ret)
  if i == 20: break
  i += 1

[('0779781015', 0.990409255027771), ('0770315020', 0.9825408458709717), ('0858856005', 0.9795480966567993), ('0695632001', 0.979347825050354)]
[('0805000007', 0.9868990182876587), ('0805000008', 0.974185585975647), ('0804992016', 0.9660730957984924), ('0905518002', 0.9493032693862915)]
[('0448509018', 0.929100513458252), ('0919365003', 0.9208903908729553), ('0448509035', 0.9195795059204102), ('0905365006', 0.9190483689308167)]
[('0685814003', 0.8507223725318909), ('0685814062', 0.8505144119262695), ('0524465001', 0.8188444375991821), ('0742258025', 0.8187606334686279)]
[('0731525009', 0.9695224165916443), ('0732409048', 0.9673168659210205), ('0844059001', 0.9658563137054443), ('0828116001', 0.965390682220459)]
[('0821417002', 0.9831748008728027), ('0837351002', 0.9814471006393433), ('0733097008', 0.9810389876365662), ('0556539018', 0.9770254492759705)]
[('0910601003', 0.9632487893104553), ('0910601001', 0.8899194598197937), ('0844483001', 0.8257529735565186), ('0910447001', 0.820083618

## articleからカテゴリーを取り出し、カスタマーの購入したカテゴリーを設定する

In [98]:
df = pd.read_csv('/content/drive/MyDrive/datas/h_and_m/articles.csv',
                 usecols= ['article_id','index_group_no','index_group_name'],
                 dtype={'article_id': 'string', 'index_group_no' : 'int' ,'index_group_name' : 'string'})

In [99]:
df.head()

,article_id,index_group_no,index_group_name
0,0108775015,1,Ladieswear
1,0108775044,1,Ladieswear
2,0108775051,1,Ladieswear
3,0110065001,1,Ladieswear
4,0110065002,1,Ladieswear


In [100]:
df['index_group_name'].unique()

<StringArray>
['Ladieswear', 'Baby/Children', 'Menswear', 'Sport', 'Divided']
Length: 5, dtype: string

In [101]:
df['index_group_no'].unique()

array([ 1,  4,  3, 26,  2])

In [117]:
article_group_dict = df.set_index('article_id')['index_group_no'].to_dict()

In [113]:
import pickle
with open('article_group_dict.pkl', 'wb') as f:
  pickle.dump(article_group_dict, f)

In [82]:
df = last.merge(df, on='article_id', how='inner')
df

,t_dat,customer_id,article_id,index_group_no,index_group_name
0,2020-09-01,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,0777148006,1,Ladieswear
1,2020-09-03,5ac5e1825104ed5fe3333e75b9337eebc4b45ad761056b...,0777148006,1,Ladieswear
2,2020-09-06,0dcf3023ea1992a78a1fcc769b6befc956f7308186496d...,0777148006,1,Ladieswear
3,2020-09-06,28b30893bbe946358103760387e3dcd09fdb7b077a942f...,0777148006,1,Ladieswear
4,2020-09-10,278f23c7fac720c2b96b25455d640860bdfa8bb3c867cf...,0777148006,1,Ladieswear
...,...,...,...,...,...
798264,2020-09-22,f71529889de7a28df0015fad0a043941ecc98883286ef0...,0737994021,4,Baby/Children
798265,2020-09-22,f79e372e21c1359dfebc7da0bf7f321d55e47b3275c351...,0533261032,4,Baby/Children
798266,2020-09-22,f82c91decd5f9abd0a7a72eae0d4911b00ed4f5b4f04f9...,0865792012,4,Baby/Children
798267,2020-09-22,f96661e9e56449885d4c4b90d3227e4abea5e0d2382e2d...,0772659001,1,Ladieswear


In [83]:
df = df.groupby('customer_id').agg({"index_group_no": set}).reset_index()

In [84]:
cust_itemgrp_dict = df.set_index('customer_id').T.to_dict('list')
cust_itemgrp_dict

In [93]:
cust_itemgrp_dict['0144aeab64b960936bf1532290e59997224b63deacf76e42c4766e6cea318b6a']

[{1, 26}]

In [96]:
import pickle
with open('customer_itemgrp_dict.pkl', 'wb') as f:
  pickle.dump(cust_itemgrp_dict, f)

In [114]:
!cp *.pkl *.json *.txt word2vec*  /content/drive/MyDrive/datas/h_and_m/trend202009

In [115]:
!ls

article_group_dict.pkl	       drive	    word2vec.model
customer_articles_202009.json  items.txt
customer_itemgrp_dict.pkl      sample_data
